# Create masks for 140W: EUC, uvel at 75m equator, vvel at 75m equator

In [1]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from xmitgcm import open_mdsdataset,utils
from MITgcmutils import rdmds, wrmds
import ecco_v4_py as ecco
import pych.ecco as pe

import warnings

In [2]:
llc = pe.read_mds('/workspace/grids/llc90/',iters=None)

/workspace/xmitgcm/xmitgcm/mds_store.py:854: UserWarning: Couldn't find available_diagnostics.log in /workspace/grids/llc90/ or /workspace/grids/llc90/. Using default version.
  "in %s or %s. Using default version." % (data_dir, grid_dir))


In [3]:
mapper = pe.atlantic_map(llc)

In [4]:
grid = ecco.get_llc_grid(llc)

### EUC Masks

In [5]:
lon = -140
lat1 = 1.5
lat2 = -1.5

In [6]:
eucMaskW,eucMaskS = pe.get_euc_masks(ds=llc,pt1=[lon,lat1],pt2=[lon,lat2],grid=grid)

/workspace/ECCOv4-py/ecco_v4_py/get_section_masks.py:51: UserWarning: 
Section name euc unavailable as pre-defined section
  warnings.warn('\nSection name %s unavailable as pre-defined section' % section_name)


In [7]:
eucMaskK = 1.*(llc.Z.where(llc.Z>-410,0.)!=0.)
llc.drF.where(eucMaskK,0.).cumsum('k').values

array([ 10.  ,  20.  ,  30.  ,  40.  ,  50.  ,  60.  ,  70.  ,  80.01,
        90.04, 100.15, 110.47, 121.27, 133.03, 146.45, 162.49, 182.31,
       207.16, 238.26, 276.68, 323.18, 378.18, 441.68, 441.68, 441.68,
       441.68, 441.68, 441.68, 441.68, 441.68, 441.68, 441.68, 441.68,
       441.68, 441.68, 441.68, 441.68, 441.68, 441.68, 441.68, 441.68,
       441.68, 441.68, 441.68, 441.68, 441.68, 441.68, 441.68, 441.68,
       441.68, 441.68])

In [8]:
k_ind = 21
eucMaskK.loc[{'k':k_ind}]=(-400-llc.Z.sel(k=k_ind))/(llc.Z.sel(k=k_ind-1)-llc.Z.sel(k=k_ind))
eucMaskK.sel(k=k_ind).values

array(0.16759494)

In [9]:
eucMaskW.sum()

<xarray.DataArray ()>
array(0)

In [10]:
eucMaskS.sum()

<xarray.DataArray ()>
array(8)

### Uvel at 75m

- 2D masks which get average between ~-.2 and .2 latitude (the two neighboring grid cells)
- 1D vertical mask grabbing the point closest to 75m

In [11]:
lon = -140
lat1 = .5
lat2 = -.5

In [12]:
uvelMaskW,uvelMaskS = pe.get_euc_masks(ds=llc,pt1=[lon,lat1],pt2=[lon,lat2],grid=grid)

/workspace/ECCOv4-py/ecco_v4_py/get_section_masks.py:51: UserWarning: 
Section name euc unavailable as pre-defined section
  warnings.warn('\nSection name %s unavailable as pre-defined section' % section_name)


In [13]:
uvelMaskW=.5*uvelMaskW
uvelMaskS=.5*uvelMaskS

In [14]:
uvelMaskS.sum()

<xarray.DataArray ()>
array(1.)

In [15]:
uvelMaskK = 1.*(llc.k.where(np.abs(llc.Z+75)<1,0.)!=0)
uvelMaskK.values

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
llc['Z'].where(uvelMaskK,0.).values

array([  0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
       -75.005,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ])

### Vvel at 75m

- 2D masks which get average between 139.5W and 140.5W at the equator
- 1D vertical mask grabbing the point closest to 75m

In [17]:
lat1 = 0#-0.2
lat2 = 0#.2
lon1 = -141
lon2 = -139

In [18]:
vvelMaskW,vvelMaskS = pe.get_euc_masks(ds=llc,pt1=[lon1,lat2],pt2=[lon2,lat2],grid=grid)

/workspace/ECCOv4-py/ecco_v4_py/get_section_masks.py:51: UserWarning: 
Section name euc unavailable as pre-defined section
  warnings.warn('\nSection name %s unavailable as pre-defined section' % section_name)


In [19]:
vvelMaskW = .5*vvelMaskW
vvelMaskS = .5*vvelMaskS

In [20]:
vvelMaskW.sum()

<xarray.DataArray ()>
array(-1.)

In [21]:
vvelMaskK = uvelMaskK.copy()

## Write them out

In [22]:
mask_dir = 'masks140W'
if not os.path.isdir(mask_dir):
    os.makedirs(mask_dir)

In [23]:
def write_llc_bin(fname,xda):
    if 'tile' in xda.dims:
        xda = xda.rename({'tile':'face'})
        
    extra_metadata = utils.get_extra_metadata(domain='llc', nx=90)
    facets = utils.rebuild_llc_facets(xda, extra_metadata)
    compact = utils.llc_facets_2d_to_compact(facets, extra_metadata)
    utils.write_to_binary(compact, fname)

In [24]:
for fld,suff in zip([eucMaskW,eucMaskS],['W','S']):
    write_llc_bin(f'{mask_dir}/eucMask{suff}',fld)
for fld,suff in zip([uvelMaskW,uvelMaskS],['W','S']):
    write_llc_bin(f'{mask_dir}/uvel_75m_mask{suff}',fld)
for fld,suff in zip([vvelMaskW,vvelMaskS],['W','S']):
    write_llc_bin(f'{mask_dir}/vvel_75m_mask{suff}',fld)
    
for fld,fbase in zip([eucMaskK,uvelMaskK,vvelMaskK],['eucMaskK','uvel_75m_maskK','vvel_75m_maskK']):
    utils.write_to_binary(fld.values.flatten(),f'{mask_dir}/{fbase}')